In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


In [ ]:
import neptune

from api_config import project_name,proxies,api_token


neptune.init('Hatespeech-CNERG/MH-Bert',api_token=api_token,proxies=proxies)

neptune.set_project('Hatespeech-CNERG/MH-Bert')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bert_codes.feature_generation import combine_features,return_dataloader
from bert_codes.data_extractor import data_collector
from bert_codes.utils import *

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=False)


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "multilingual_bert/", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()


In [ ]:
import glob 


train_files=glob.glob('full_data/Train/*.csv')
test_files=glob.glob('full_data/Test/*.csv')
val_files=glob.glob('full_data/Val/*.csv')

In [ ]:
train_files[2].split('/')[-1][:-12]

In [ ]:
df=data_collector(train_files,'Polish',is_train=True,sample_ratio=20,type_train='baseline')


In [ ]:
list_ele= [1/ele for ele in df.label.value_counts()]
print(list_ele)

In [ ]:
params={
    'is_model':False,
    'files':test_files,
    'path_files':'multilingual_bert/',
    'language':'English',
    ''
}

In [ ]:
def Eval_phase(model=None,test_files,language,is_model=True,path_files='multilingual_bert/'):
    
    if(is_model=True):
        model.eval()
    else:
        model = BertForSequenceClassification.from_pretrained(
            path_files, # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
        )
        model.eval()

        
    df_test=data_collector(test_files,language,is_train=False,sample_ratio=100,type_train='baseline')
    sentences_test = df_test.text.values
    labels_test = df_test.label.values
    input_test_ids,att_masks_test=combine_features(sentences_test,tokenizer)
    test_dataloader=return_dataloader(input_test_ids,labels_test,att_masks_test,batch_size=8)
    print("Running Test...")
    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    true_labels=[]
    pred_labels=[]
    # Evaluate data for one epoch
    for batch in test_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        pred_labels+=list(np.argmax(logits, axis=1).flatten())
        true_labels+=list(label_ids.flatten())

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print(" Fscore: {0:.2f}".format(f1_score(true_labels, pred_labels, average='macro')))
    print(" Test took: {:}".format(format_time(time.time() - t0)))

    
    return f1_score(true_labels, pred_labels, average='macro')

In [ ]:
list_lang=[]
for file in train_files:
    list_lang.append(file.split('/')[-1][:-12])
list_lang=list(set(list_lang))

In [ ]:
for lang in list_lang:
    print(lang)
    Eval_phase(model,lang)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig

import random
batch_size=8
def model_lang_specific(language,sampling_percent=0.1):

    # Load the BERT tokenizer.
    
    # Load BertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = BertForSequenceClassification.from_pretrained(
        "multilingual_bert/", # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )

    # Tell pytorch to run this model on the GPU.
    model.cuda()

    
#     df_train=[]
#     for file in train_files:
#         lang_temp=file.split('/')[-1][:-12]
#         if(lang_temp==language):
#             df_train.append(pd.read_csv(file))
#     df_train=pd.concat(df_train,axis=0)
    
#     df_train=stratified_sample_df(df_train, 'label', int(len(df_train)*sampling_percent/100)+1)
    
    df_train=data_collector(train_files,language,is_train=True,sample_ratio=sampling_percent,type_train='baseline')
    df_val=data_collector(val_files,language,is_train=False,sample_ratio=sampling_percent,type_train='baseline')
    
    
    print(len(df_train))
    
    sentences_train = df_train.text.values
    labels_train = df_train.label.values
    sentences_val = df_val.text.values
    labels_val = df_val.label.values
    sentences_test = df_test.text.values
    
    
    
    input_train_ids,att_masks_train=combine_features(sentences_train,tokenizer)
    input_val_ids,att_masks_val=combine_features(sentences_val,tokenizer)
    print(att_masks_train[0:5])
    
    
    train_dataloader = return_dataloader(input_train_ids,labels_train,att_masks_train)
    validation_dataloader=return_dataloader(input_val_ids,labels_val,att_masks_val)
    
    
    train_inputs = torch.tensor(input_train_ids)
    validation_inputs = torch.tensor(input_val_ids)

    train_labels = torch.tensor(labels_train,dtype=torch.long)
    validation_labels = torch.tensor(labels_val,dtype=torch.long)

    train_masks = torch.tensor(att_masks_train)
    validation_masks = torch.tensor(att_masks_val)
    print('attention_masks',train_masks.shape)
    
    
    # Create the DataLoader for our training set.
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create the DataLoader for our validation set.
    validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
    
    
    optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
    
    # Number of training epochs (authors recommend between 2 and 4)
    epochs = 5

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
   
    # This training code is based on the `run_glue.py` script here:
    # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


    # Set the seed value all over the place to make this reproducible.
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # Store the average loss after each epoch so we can plot them.
    loss_values = []

    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
#                 print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple.
            loss = outputs[0]

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

#         print("")
#         print("  Average training loss: {0:.2f}".format(avg_train_loss))
#         print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

#         # ========================================
#         #               Validation
#         # ========================================
#         # After the completion of each training epoch, measure our performance on
#         # our validation set.

#         print("")
#         print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        true_labels=[]
        pred_labels=[]
        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)

            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch

            # Telling the model not to compute or store gradients, saving memory and
            # speeding up validation
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)

            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = outputs[0]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            pred_labels+=list(np.argmax(logits, axis=1).flatten())
            true_labels+=list(label_ids.flatten())
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy

            # Track the number of batches
            nb_eval_steps += 1
        fscore=Eval_phase(model,language)
        # Report the final accuracy for this validation run.
#         print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
#         print("  Fscore: {0:.2f}".format(f1_score(true_labels, pred_labels, average='macro')))
#         print("  Validation took: {:}".format(format_time(time.time() - t0)))

#     print("")
#     print("Training complete!")
    fscore=Eval_phase(model,language)
    del model
    return fscore

In [ ]:
for lang in list_lang:
    print(lang)
    model_lang_specific(lang,sampling_percent=1)

In [ ]:
dict_language={}
for lang in ['Portugese']:
    result_avg=[]
    for sp in [20]:
        torch.cuda.empty_cache()
        print(sp,"percent")
        avg_fscore=0
        for i in range(5):
            avg_fscore+=model_lang_specific(lang,sampling_percent=sp)
        avg_fscore=avg_fscore/5
        result_avg.append((sp,avg_fscore))
    dict_language[lang]=result_avg

In [ ]:
Fscore ----- 0.30 (0.5),0.46(1),0.55(5),0.6(10),0.64(20)

In [ ]:
import matplotlib.pyplot as plt
for key in dict_language:
    x=[ele[0] for ele in dict_language[key]]
    y=[ele[1] for ele in dict_language[key]]
    plt.plot(x,y,key)
    
plt.legend()
plt.show()  

In [ ]:
list_lang

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)


In [ ]:
tokenizer

In [ ]:
tokenizer.save_pretrained('multilingual_bert/')


In [ ]:
import logging
import math
import os

import torch
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

from transformers.configuration_bert import BertConfig
#from transformers.file_utils import add_start_docstrings, add_start_docstrings_to_callable
from transformers.modeling_utils import PreTrainedModel, prune_linear_layer



In [ ]:
from transformers.modeling_bert import *

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
labels_train = df_train.label.value_counts


In [34]:
loss = nn.CrossEntropyLoss()
input = torch.ones(5, 1, requires_grad=True)
target = torch.empty(1, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()



#torch.nn.functional.one_hot(tensor, num_classes=-1)

ValueError: Expected input batch_size (5) to match target batch_size (1).

In [35]:
output

tensor(1.6094, grad_fn=<NllLossBackward>)

In [33]:
input

tensor([[1., 1., 1., 1., 1.]], requires_grad=True)